In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
#df= pd.read_csv('../../data/more_features/with_correlated_feature_0.1.csv')
df =pd.read_csv('../../data/more_features/more_feats_correlated_missing_3.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,-1.000000,316.593436,-1.000000,-1.000000,3.531295,1,-1.000000,-1.000000,96.721542,49.530542,23.186628,-1.0,-1.000000,57.247868,Hemolytic anemia
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,125.249617,-1.000000,1,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.0,-1.000000,-1.000000,No anemia
2,12.749357,-1.000000,-1.000000,6.179371,498.418768,76.759285,-1.000000,4.982859,1,1.022939,-1.000000,75.739552,72.072041,20.600875,-1.0,38.248071,32.068372,Iron deficiency anemia
3,11.508870,197.180945,1.200125,-1.000000,457.033309,102.900301,131.177927,3.355346,1,-1.000000,-1.000000,66.999185,18.353272,-1.000000,-1.0,34.526609,28.702050,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,2.713693,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.369968,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
utils.get_dt_performance(df)

(0.7770714285714285,
 0.7458343221135431,
 0.8570010394453483,
 datetime.timedelta(microseconds=2000))

In [5]:
df.label.value_counts()

No anemia                               10000
Anemia of chronic disease                9756
Iron deficiency anemia                   9267
Unspecified anemia                       9033
Aplastic anemia                          9020
Vitamin B12/Folate deficiency anemia     9000
Hemolytic anemia                         8976
Inconclusive diagnosis                   4948
Name: label, dtype: int64

In [6]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [7]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [ ]:
for steps in [int(7e6), int(8e6), int(9e6), int(10e6), int(10.5e6), int(11e6)]:
#for steps in [int(2e3)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_missing_3_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.11     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.6      |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 1042     |
|    time_elapsed     | 282      |
|    total_timesteps  | 294604   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 3.59     |
|    n_updates        | 61150    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.86     |
|    ep_rew_mean      | -0.9     |
|    exploration_rate | 0.161    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 895      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.63     |
|    ep_rew_mean      | -0.04    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.55     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 656      |
|    time_elapsed     | 8247     |
|    total_timesteps  | 5416729  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.17     |
|    n_updates        | 1341682  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.79     |
|    ep_rew_mean      | -0.02    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.53     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 640      |
|    time_elapsed     | 9024     |
|    total_timesteps  | 5782949  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.65     |
|    ep_rew_mean      | -0.4     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.31     |
| time/               |          |
|    episodes         | 1100000  |
|    fps              | 590      |
|    time_elapsed     | 6009     |
|    total_timesteps  | 3546357  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0687   |
|    n_updates        | 874089   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.31     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.47     |
| time/               |          |
|    episodes         | 1200000  |
|    fps              | 587      |
|    time_elapsed     | 6613     |
|    total_timesteps  | 3886202  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.31     |
|    ep_rew_mean      | -0.96    |
|    exploration_rate | 0.356    |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 748      |
|    time_elapsed     | 815      |
|    total_timesteps  | 610264   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 13.4     |
|    n_updates        | 140065   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.93     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.05     |
| time/               |          |
|    episodes         | 300000   |
|    fps              | 711      |
|    time_elapsed     | 1315     |
|    total_timesteps  | 935497   |
| train/              |          |
|    learning_rate  

In [14]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_missing_2_11500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,5.0,0.0,1.0,1.0,0.0,"[gender, hemoglobin, mcv, ret_count, Aplastic ...",6.0,6.0
1,4.0,1.0,1.0,1.0,0.0,"[gender, hemoglobin, mcv, Unspecified anemia]",2.0,2.0
2,4.0,2.0,1.0,1.0,0.0,"[gender, hemoglobin, mcv, Unspecified anemia]",2.0,2.0
3,5.0,3.0,1.0,1.0,0.0,"[gender, hemoglobin, mcv, ret_count, Hemolytic...",5.0,5.0
4,5.0,4.0,1.0,1.0,0.0,"[gender, hemoglobin, mcv, ret_count, Hemolytic...",5.0,5.0


In [15]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [16]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [17]:
test_df.y_pred.value_counts()

7.0    4649
2.0    3037
0.0    2021
3.0    1191
5.0    1117
6.0    1115
4.0     870
Name: y_pred, dtype: int64

In [18]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

58.67142857142858

In [19]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.450642857142857, 0.07628571428571429)

In [20]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



(0.5867142857142857, 0.5801727360556713, 0.7651865385056864)

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)